In [2]:
%run __common.py
%load_ext autoreload
%autoreload 2

Provided checkpoint directory /iopsstor/scratch/cscs/stefschu/DSM500/github/analysis/checkpoints does not exist, skipping load


Loaded validation datapipe of size 6


In [13]:
dp_orig = ERA5HDF5Datapipe(
    data_dir=to_absolute_path(os.path.join(cfg.dataset_path, "train")),
    stats_dir=to_absolute_path(os.path.join(cfg.dataset_path, "stats")),
    channels=channels_list,
    latlon_resolution=cfg.latlon_res,
    interpolation_type=None,
    num_samples_per_year=cfg.num_samples_per_year_train,
    num_steps=1,
    num_history=cfg.num_history,
    use_cos_zenith=cfg.use_cos_zenith,
    use_time_of_year_index=cfg.use_time_of_year_index,
    cos_zenith_args={
            "dt": cfg.dt,
            "start_year": cfg.start_year,
        },
    batch_size=1,
    num_workers=cfg.num_workers,
    device=dist.device,
    process_rank=dist.rank,
    world_size=dist.world_size,
)

data = next(iter(dp_orig))

invar = data[0]["invar"]
outvar = data[0]["outvar"]
try:
    cos_zenith = data[0]["cos_zenith"]
except KeyError:
    cos_zenith = None
try:
    time_idx = data[0]["time_of_year_idx"].item()
except KeyError:
    time_idx = None

invar_cat = prepare_input(
    invar,
    cos_zenith,
    num_history=cfg.num_history,
    static_data=trainer.static_data,
    step=1,
    time_idx=time_idx,
    stride=cfg.stride,
    dt=cfg.dt,
    num_samples_per_year=cfg.num_samples_per_year_train,
    device=dist.device,
)
invar_cat, outvar = invar_cat.to(dtype=trainer.dtype), outvar.to(dtype=trainer.dtype)

print(invar_cat.shape, invar_cat.dtype, outvar.shape, outvar.dtype)


trainer.train(invar_cat, outvar)

/usr/local/lib/python3.10/dist-packages/nvidia/dali/pipeline.py:912: Warning: The external source node '<modulus.datapipes.climate.era5_hdf5.ERA5DaliExternalSource object at 0xfff4e1865780>' produces 4 outputs, but the outputs at the indices 2, 3 are not used. For best performance, adjust your callback so that it computes only the needed outputs.
  warnings.warn(


torch.Size([1, 21, 721, 1440]) torch.bfloat16 torch.Size([1, 1, 21, 721, 1440]) torch.bfloat16


tensor(1.1875, device='cuda:0', dtype=torch.bfloat16, grad_fn=<AddBackward0>)

In [3]:
num_rollout_steps = 2

dp_updated = ERA5HDF5Datapipe(
    data_dir=os.path.join(cfg.dataset_path, "train"),
    stats_dir=os.path.join(cfg.dataset_path, "stats"),
    channels=channels_list,
    latlon_resolution=cfg.latlon_res,
    interpolation_type=None,
    num_samples_per_year=cfg.num_samples_per_year_train,
    num_steps=num_rollout_steps,
    num_history=cfg.num_history,
    use_cos_zenith=cfg.use_cos_zenith,
    use_time_of_year_index=cfg.use_time_of_year_index,
    cos_zenith_args={
        "dt": cfg.dt,
        "start_year": cfg.start_year,
    },
    batch_size=1,
    num_workers=cfg.num_workers,
    device=dist.device,
    process_rank=dist.rank,
    world_size=dist.world_size,
)

data = next(iter(dp_updated))

invar = data[0]["invar"]
outvar = data[0]["outvar"]
try:
    cos_zenith = data[0]["cos_zenith"]
except KeyError:
    cos_zenith = None
try:
    time_idx = data[0]["time_of_year_idx"].item()
except KeyError:
    time_idx = None

invar_cat = prepare_input(
    invar,
    cos_zenith,
    num_history=cfg.num_history,
    static_data=trainer.static_data,
    step=1,
    time_idx=time_idx,
    stride=cfg.stride,
    dt=cfg.dt,
    num_samples_per_year=cfg.num_samples_per_year_train,
    device=dist.device,
)
invar_cat, outvar = invar_cat.to(dtype=trainer.dtype), outvar.to(dtype=trainer.dtype)

print(invar_cat.shape, outvar.shape)

trainer.train(invar_cat, outvar)

/usr/local/lib/python3.10/dist-packages/nvidia/dali/pipeline.py:912: Warning: The external source node '<modulus.datapipes.climate.era5_hdf5.ERA5DaliExternalSource object at 0xfffba5713e20>' produces 4 outputs, but the outputs at the indices 2, 3 are not used. For best performance, adjust your callback so that it computes only the needed outputs.
  warnings.warn(


torch.Size([1, 21, 721, 1440]) torch.Size([1, 2, 21, 721, 1440])


tensor(2.3125, device='cuda:0', dtype=torch.bfloat16, grad_fn=<AddBackward0>)

Process SpawnProcess-8:
Process SpawnProcess-4:
Process SpawnProcess-3:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/worker.py", line 462, in worker
    scheduled, shm_chunk = worker_context.get_task()
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/worker.py", line 462, in worker
    scheduled, shm_chunk = worker_context.get_task()
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/worker.py", line 434, in get_task
    sched